In [196]:
from scipy import stats
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder,OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import lazypredict
from lazypredict.Supervised import LazyRegressor
from sklearn.model_selection import GridSearchCV

Đọc và chia data

In [197]:
# data = pd.read_csv("dataAfterCleaningBinhTan.csv", delimiter=",")
data = data[~data['Gia/m2'].isin(a)]
data.head()

,chieuDai,chieuNgang,dienTich,Gia/m2,Phongngu,SoTang,PhongTam,Loai,GiayTo,TinhTrangNoiThat,Phuong
0,16.00,4.00,65.00,1570.42,2,2.00,2.00,"nhà ngõ, hẻm",đã có sổ,nội thất đầy đủ,phường bình hưng hoà b
1,15.00,4.00,60.00,3465.42,3,3.00,3.00,"nhà ngõ, hẻm",đã có sổ,nội thất cao cấp,phường bình trị đông a
3,18.00,4.00,72.00,3350.83,5,4.00,5.00,"nhà ngõ, hẻm",đã có sổ,nội thất cao cấp,phường bình trị đông a
4,8.12,8.12,66.00,3775.42,4,4.00,4.00,"nhà ngõ, hẻm",đã có sổ,NaN,phường bình hưng hòa
5,8.00,3.00,23.00,4982.08,2,3.00,2.00,"nhà ngõ, hẻm",đã có sổ,nội thất cao cấp,phường bình hưng hoà a


In [198]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5618 entries, 0 to 6715
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   chieuDai          5618 non-null   float64
 1   chieuNgang        5618 non-null   float64
 2   dienTich          5618 non-null   float64
 3   Gia/m2            5618 non-null   float64
 4   Phongngu          5618 non-null   int64  
 5   SoTang            5618 non-null   float64
 6   PhongTam          5618 non-null   float64
 7   Loai              5618 non-null   object 
 8   GiayTo            5609 non-null   object 
 9   TinhTrangNoiThat  3648 non-null   object 
 10  Phuong            5618 non-null   object 
dtypes: float64(6), int64(1), object(4)
memory usage: 526.7+ KB


In [199]:
target = "Gia/m2"
x = data.drop(target, axis=1)
y = data[target]
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=1609)

In [200]:
print("train.shape: ",x_train.shape)
print("test.shape: ",x_test.shape)

train.shape:  (4494, 10)
test.shape:  (1124, 10)


Xem unique của từng cột categorical

In [201]:
print("uniqueLoai:")
x_train.Loai.unique()

uniqueLoai:


array(['nhà ngõ, hẻm', 'nhà mặt phố, mặt tiền', 'nhà phố liền kề',
       'nhà biệt thự'], dtype=object)

In [202]:
print("uniqueGiayTo:")
x_train.GiayTo.unique()

uniqueGiayTo:


array(['đã có sổ', 'đang chờ sổ', 'sổ chung / công chứng vi bằng',
       'giấy tờ viết tay', 'không có sổ', nan], dtype=object)

In [203]:
print("uniqueTinhTrangNoiThat:")
x_train.TinhTrangNoiThat.unique()

uniqueTinhTrangNoiThat:


array([nan, 'nội thất đầy đủ', 'hoàn thiện cơ bản', 'nội thất cao cấp',
       'bàn giao thô'], dtype=object)

In [204]:
print("uniquePhuong:")
x_train.Phuong.unique()

uniquePhuong:


array(['phường bình hưng hòa', 'phường bình trị đông',
       'phường bình hưng hoà a', 'phường bình hưng hoà b',
       'phường bình trị đông b', 'phường tân tạo a', 'phường tân tạo',
       'phường an lạc', 'phường bình trị đông a', 'phường an lạc a'],
      dtype=object)

In [205]:
## lấy ra list Phường theo thứ tự tăng dần của giá
df = pd.DataFrame(data)

def sorted_districts(col):
    table = df.groupby(col)['Gia/m2'].mean().reset_index()
  
    sorted_table = table.sort_values(by='Gia/m2')
    sorted_districts = sorted_table[col].tolist()
    return sorted_districts

list_sorted_Phuong = sorted_districts('Phuong')

In [206]:
for i in range(0, len(list_sorted_Phuong), 4):
    print(list_sorted_Phuong[i:i+4])

['phường tân tạo a', 'phường bình hưng hoà b', 'phường tân tạo', 'phường bình hưng hòa']
['phường bình trị đông a', 'phường bình trị đông', 'phường bình hưng hoà a', 'phường an lạc']
['phường an lạc a', 'phường bình trị đông b']


Tiền xử lý lần 2

In [207]:
Loai_values = ["unknown",'nhà ngõ, hẻm', 'nhà phố liền kề', 'nhà mặt phố, mặt tiền', 'nhà biệt thự']
GiayTo_values = ["unknown",'giấy tờ viết tay', 'không có sổ', 'đang chờ sổ', 
            'sổ chung / công chứng vi bằng', 'đã có sổ']
TinhTrangNoiThat_values = ["unknown",'bàn giao thô', 'hoàn thiện cơ bản', 'nội thất đầy đủ', 'nội thất cao cấp']


ord_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value="unknown")),
    ("encoder", OrdinalEncoder(categories=[Loai_values,GiayTo_values,TinhTrangNoiThat_values,list_sorted_Phuong]))
])

num_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])
preprocessor = ColumnTransformer(transformers=[
    ("num_features", num_transformer, ["chieuDai", "chieuNgang","dienTich","Phongngu","SoTang","PhongTam"]),
    ("ordinal_features", ord_transformer, ["Loai","GiayTo","TinhTrangNoiThat","Phuong"]),
])



In [208]:
# x_train_preprocessor = preprocessor.fit_transform(x_train)
# x_test_preprocessor = preprocessor.transform(x_test)
# lazy_reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None )
# models, predictions = lazy_reg.fit(x_train_preprocessor, x_test_preprocessor, y_train, y_test)
# print(predictions)

In [ ]:
from sklearn.linear_model import PoissonRegressor,LinearRegression,Ridge,Lasso
from sklearn.tree import DecisionTreeRegressor,ExtraTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor,RandomForestRegressor,HistGradientBoostingRegressor 
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
import xgboost as xgb
from joblib import dump, load
reg = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor",RandomForestRegressor()),
])
reg.fit(x_train, y_train)

# Lưu model
# dump(reg, 'model1.joblib')


In [210]:
# parameters = {
#     "regressor__n_estimators": [50, 100, 200, 500],
#     "regressor__criterion": ["squared_error", "absolute_error", "poisson"],
#     "regressor__max_depth": [None, 5, 10, 20],
#     "regressor__max_features": ["sqrt", "log2"],
#     "preprocessor__num_features__imputer__strategy": ["mean", "median"],
# }
# model = GridSearchCV(reg, param_grid=parameters, scoring="r2", cv=6, verbose=2, n_jobs=-1)
# model.fit(x_train, y_train)
# print(model.best_score_)
# print(model.best_params_)

In [211]:
y_predict = reg.predict(x_test)

In [212]:
a = []
accuracy_list = [(i, j, abs(i - j) / i * 100 if i != 0 else 0) for i, j in zip(y_test, y_predict)]

accuracy_list_sorted = sorted(accuracy_list, key=lambda x: x[2])

for actual, predict, percentage_error in accuracy_list_sorted:
    if(percentage_error > 50):
        a.append(actual)
    print("Actual: {} - Predict: {} - Percentage Error: {:.2f}%".format(actual, predict, percentage_error))

Actual: 5681.666666666667 - Predict: 5679.209375000002 - Percentage Error: 0.04%
Actual: 3977.0833333333335 - Predict: 3979.5815972222213 - Percentage Error: 0.06%
Actual: 4398.333333333333 - Predict: 4401.158333333332 - Percentage Error: 0.06%
Actual: 5625.0 - Predict: 5628.727777777776 - Percentage Error: 0.07%
Actual: 4327.083333333333 - Predict: 4323.323611111113 - Percentage Error: 0.09%
Actual: 3949.5833333333335 - Predict: 3946.071527777778 - Percentage Error: 0.09%
Actual: 4143.333333333333 - Predict: 4147.078541666665 - Percentage Error: 0.09%
Actual: 4411.666666666667 - Predict: 4415.749999999998 - Percentage Error: 0.09%
Actual: 3645.8333333333335 - Predict: 3649.4583333333308 - Percentage Error: 0.10%
Actual: 3645.8333333333335 - Predict: 3642.0958333333315 - Percentage Error: 0.10%
Actual: 5000.0 - Predict: 4992.731249999999 - Percentage Error: 0.15%
Actual: 3794.1666666666665 - Predict: 3800.4274206349232 - Percentage Error: 0.17%
Actual: 4398.333333333333 - Predict: 4390

In [213]:

# Khởi tạo các khoảng lỗi
error_ranges = [(0, 20), (20, 50), (50, 70),(70,100),(100, float('inf'))]

# Khởi tạo danh sách để lưu kết quả đếm số lượng trong mỗi khoảng
range_counts = {f"{low}-{high}": 0 for low, high in error_ranges}

# Đếm số lượng dự đoán thuộc vào từng khoảng lỗi
for _, _, percentage_error in accuracy_list_sorted:
    for low, high in error_ranges:
        if low <= percentage_error < high:
            range_counts[f"{low}-{high}"] += 1
            break

# Tổng số lượng dự đoán
total_predictions = len(accuracy_list_sorted)

# Tính và in phần trăm cho mỗi khoảng lỗi
print("Percentage Error Ranges:")
for range_str, count in range_counts.items():
    percentage = (count / total_predictions) * 100
    print(f"Error Range {range_str}%: {count} predictions ({percentage:.2f}%)")


Percentage Error Ranges:
Error Range 0-20%: 916 predictions (81.49%)
Error Range 20-50%: 198 predictions (17.62%)
Error Range 50-70%: 8 predictions (0.71%)
Error Range 70-100%: 1 predictions (0.09%)
Error Range 100-inf%: 1 predictions (0.09%)


In [214]:
import math
print("MAE {}".format(mean_absolute_error(y_test, y_predict)))
print("RMSE {}".format(math.sqrt(mean_squared_error(y_test, y_predict))))
print("R2 {}".format(r2_score (y_test, y_predict)))

MAE 447.0886786019718
RMSE 637.9891208998217
R2 0.6687769906474662
